In [7]:
import pandas as pd
from io import StringIO

import time
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome() 

data_di = '12/07/2023'

url = f'''https://www2.bmf.com.br/pages/portal/bmfbovespa/boletim1/SistemaPregao1.asp?
pagetype=pop&caminho=Resumo%20Estat%EDstico%20-%20Sistema%20Preg%E3o&Data={data_di}
&Mercadoria=DI1'''

driver.get(url)


In [2]:
path_tabela = '/html/body/div/div[2]/form[1]/table[3]/tbody/tr[3]/td[3]/table'
elemento = driver.find_element(By.XPATH, path_tabela)
html_tabela = elemento.get_attribute('outerHTML')
tabela = pd.read_html(StringIO(html_tabela))[0]
tabela.head()

,0,1,2,3,4,5,6,7,8,9,10
0,AJUSTE ANTER. (3),AJUSTE CORRIG. (4),PREÇO ABERT.,PREÇO MÍN.,PREÇO MÁX.,PREÇO MÉD.,ÚLT. PREÇO,AJUSTE,VAR. PTOS.,ÚLT. OF. COMPRA,ÚLT. OF. VENDA
1,"99.291,76","99.291,76",13650,13648,13652,13651,13650,"99.291,62","0,14-",13648,13652
2,"98.161,00","98.161,00",13450,13450,13470,13469,13470,"98.161,83","0,83+",13464,13470
3,"97.203,24","97.203,24",13364,13340,13368,13353,13358,"97.204,81","1,57+",13358,13360
4,"96.231,22","96.231,22",13200,13200,13220,13208,13215,"96.231,10","0,12-",13210,13235


In [3]:
path_indice = '/html/body/div/div[2]/form[1]/table[3]/tbody/tr[3]/td[1]/table'        
elemento_indice = driver.find_element(By.XPATH, path_indice)
html_indice = elemento_indice.get_attribute('outerHTML')
indice = pd.read_html(StringIO(html_indice))[0]
indice.head()

,0
0,VENCTO
1,Q23
2,U23
3,V23
4,X23


In [4]:
legenda = pd.Series(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
                        index = ['F', 'G', 'H', 'J', 'K', 'M', 'N', 'Q', 'U', 'V', 'X', 'Z'])

tabela.columns = tabela.loc[0]
tabela = tabela['ÚLT. PREÇO']
tabela = tabela.drop(0, axis = 0)

indice.columns = indice.loc[0]
indice = indice.drop(0, axis = 0)
tabela.index = indice['VENCTO']

tabela = tabela.astype(int)
tabela = tabela[tabela != 0]
tabela = tabela/1000

lista_datas = []

for indice in tabela.index:
        letra = indice[0]
        ano = indice[1:3]
        mes = legenda[letra]
        data = f"{mes}-{ano}"
        data = datetime.strptime(data, "%b-%y")
        lista_datas.append(data)
     
tabela.index = lista_datas  

tabela = tabela/100

tabela

2023-08-01    0.13650
2023-09-01    0.13470
2023-10-01    0.13358
2023-11-01    0.13215
2023-12-01    0.13030
2024-01-01    0.12850
2024-02-01    0.12635
2024-03-01    0.12485
2024-04-01    0.12305
2024-05-01    0.12070
2024-06-01    0.11905
2024-07-01    0.11720
2024-10-01    0.11205
2025-01-01    0.10830
2025-04-01    0.10585
2025-07-01    0.10385
2025-10-01    0.10285
2026-01-01    0.10175
2026-04-01    0.10160
2026-07-01    0.10160
2026-10-01    0.10170
2027-01-01    0.10175
2027-04-01    0.10155
2027-07-01    0.10265
2027-10-01    0.10225
2028-01-01    0.10340
2028-04-01    0.10370
2029-01-01    0.10470
2030-01-01    0.10550
2031-01-01    0.10640
2032-01-01    0.10640
2033-01-01    0.10680
Name: ÚLT. PREÇO, dtype: float64

In [5]:
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "vscode"

fig= go.Figure()

fig.add_trace(go.Scatter(x=tabela.index, y=tabela.values, name=f"Curva de Juros",
                        line=dict(color='royalblue', width=2), mode='lines+markers'))

fig.update_layout(yaxis=dict(tickformat=".1%", tickfont=dict(color="black")),
                            xaxis=dict(tickfont=dict(color="black")))

fig.show()